Bringing a Gun to a Trainer Fight
=================================

Uh-oh -- you've been cornered by one of Commander Lambdas elite bunny trainers! Fortunately, you grabbed a beam weapon from an abandoned storeroom while you were running through the station, so you have a chance to fight your way out. But the beam weapon is potentially dangerous to you as well as to the bunny trainers: its beams reflect off walls, meaning you'll have to be very careful where you shoot to avoid bouncing a shot toward yourself!

Luckily, the beams can only travel a certain maximum distance before becoming too weak to cause damage. You also know that if a beam hits a corner, it will bounce back in exactly the same direction. And of course, if the beam hits either you or the bunny trainer, it will stop immediately (albeit painfully). 

Write a function solution(dimensions, your_position, trainer_position, distance) that gives an array of 2 integers of the width and height of the room, an array of 2 integers of your x and y coordinates in the room, an array of 2 integers of the trainer's x and y coordinates in the room, and returns an integer of the number of distinct directions that you can fire to hit the elite trainer, given the maximum distance that the beam can travel.

The room has integer dimensions [1 < x_dim <= 1250, 1 < y_dim <= 1250]. You and the elite trainer are both positioned on the integer lattice at different distinct positions (x, y) inside the room such that [0 < x < x_dim, 0 < y < y_dim]. Finally, the maximum distance that the beam can travel before becoming harmless will be given as an integer 1 < distance <= 10000.

For example, if you and the elite trainer were positioned in a room with dimensions [3, 2], your_position [1, 1], trainer_position [2, 1], and a maximum shot distance of 4, you could shoot in seven different directions to hit the elite trainer (given as vector bearings from your location): [1, 0], [1, 2], [1, -2], [3, 2], [3, -2], [-3, 2], and [-3, -2]. As specific examples, the shot at bearing [1, 0] is the straight line horizontal shot of distance 1, the shot at bearing [-3, -2] bounces off the left wall and then the bottom wall before hitting the elite trainer with a total shot distance of sqrt(13), and the shot at bearing [1, 2] bounces off just the top wall before hitting the elite trainer with a total shot distance of sqrt(5).

Languages
=========

To provide a Java solution, edit Solution.java

To provide a Python solution, edit solution.py

Test cases
==========
Your code should pass the following test cases.
Note that it may also be run against hidden test cases not shown here.

-- Java cases --

Input:

Solution.solution([3,2], [1,1], [2,1], 4)

Output:
    7

Input:

Solution.solution([300,275], [150,150], [185,100], 500)

Output:
    9

-- Python cases --

Input:

solution.solution([3,2], [1,1], [2,1], 4)

Output:
    7

Input:

solution.solution([300,275], [150,150], [185,100], 500)

Output:
    9

In [3]:
import math

def mirror_positive(size_x, size_y, mine_x, mine_y, trainer_x, trainer_y, num_mirrors_x, num_mirrors_y):
    mine_mirrored_coords = []
    trainer_mirrored_coords = []
    for i in range(num_mirrors_x):
        for j in range(num_mirrors_y):
            if i % 2 == 0:
                mine_mirrored_x = i * size_x + mine_x
                trainer_mirrored_x = i * size_x + trainer_x
            else:
                mine_mirrored_x = (i + 1) * size_x - mine_x
                trainer_mirrored_x = (i + 1) * size_x - trainer_x
            if j % 2 == 0:
                mine_mirrored_y = j * size_y + mine_y
                trainer_mirrored_y = j * size_y + trainer_y
            else:
                mine_mirrored_y = (j + 1) * size_y - mine_y
                trainer_mirrored_y = (j + 1) * size_y - trainer_y

            mine_mirrored_coords.append((mine_mirrored_x, mine_mirrored_y))
            trainer_mirrored_coords.append((trainer_mirrored_x, trainer_mirrored_y))

    return mine_mirrored_coords, trainer_mirrored_coords

def mirror_negative(coords):
    mirrored = []
    for (x, y) in coords:
        mirrored.append((x, y))
        mirrored.append((-x, y))
        mirrored.append((x, -y))
        mirrored.append((-x, -y))
    return mirrored

def check_direct_shot(mine_x, mine_y, mirrored_coords, distance):
    mirrored_coords_ok = []
    for coords in mirrored_coords:
        delta_x = abs(mine_x - coords[0])
        delta_y = abs(mine_y - coords[1])
        if distance >= math.sqrt(delta_x ** 2 + delta_y ** 2):
            mirrored_coords_ok.append(coords)
    return mirrored_coords_ok

def filter_unique_angles(x, y, mirrored_coords):
    angles = {}
    for mirrored_coord in mirrored_coords:
        x_mirrored, y_mirrored = mirrored_coord
        delta_x = x_mirrored - x
        delta_y = y_mirrored - y
        if delta_x == 0 and delta_y == 0:
            continue
        angle = math.atan2(delta_y, delta_x)
        distance = math.sqrt(delta_x ** 2 + delta_y ** 2)
        if angle in angles:
            if distance < angles[angle]:
                angles[angle] = distance
        else:
            angles[angle] = distance
    return angles

def solution(dimensions, mine_position, trainer_position, distance):
    size_x, size_y = dimensions
    assert 1 < size_x <= 1250
    assert 1 < size_y <= 1250

    mine_x, mine_y = mine_position
    trainer_x, trainer_y = trainer_position

    assert 0 < mine_x < size_x
    assert 0 < mine_y < size_y

    assert 0 < trainer_x < size_x
    assert 0 < trainer_y < size_y

    assert 1 < distance <= 10000

    if mine_position == trainer_position:
        return 0

    # calculate maximum number of mirror order for x
    max_mirrors_x = int(math.ceil(float(mine_x + distance) / size_x))

    # calculate maximum number of mirror order for y
    max_mirrors_y = int(math.ceil(float(mine_y + distance) / size_y))

    # mirror to 1st quadrant
    mine_mirrored_coords, trainer_mirrored_coords = mirror_positive(size_x, size_y, mine_x, mine_y,
                                                                    trainer_x, trainer_y, max_mirrors_x, max_mirrors_y)

    # mirror to 2-4 quadrants
    mine_mirrored_coords = mirror_negative(mine_mirrored_coords)
    trainer_mirrored_coords = mirror_negative(trainer_mirrored_coords)

    # filter by distance
    mine_mirrored_coords = check_direct_shot(mine_x, mine_y, mine_mirrored_coords, distance)
    trainer_mirrored_coords = check_direct_shot(mine_x, mine_y, trainer_mirrored_coords, distance)

    # filter by angles
    my_angles = filter_unique_angles(mine_x, mine_y, mine_mirrored_coords)
    trainer_angles = filter_unique_angles(mine_x, mine_y, trainer_mirrored_coords)

    # calculate directions
    directions = 0
    for angle in trainer_angles:
        if angle not in my_angles or my_angles[angle] > trainer_angles[angle]:
            directions += 1
    return directions

In [4]:
# Tests
tests = (([3, 2], [1, 1], [2, 1], 4), 7), \
        (([3, 2], [1, 1], [2, 1], 2), 1), \
        (([3, 2], [1, 1], [1, 1], 2), 0), \
        (([10, 10], [1, 1], [9, 9], 100), 277), \
        (([10, 10], [1, 1], [9, 9], 12), 1), \
        (([300, 275], [150, 150], [185, 100], 500), 9), \
        (([300, 275], [150, 150], [185, 185], 500), 8)

for test in tests:
    input, output = test
    print(input)
    sol = solution(*input)
    print(sol)
    assert sol == output

([3, 2], [1, 1], [2, 1], 4)
7
([3, 2], [1, 1], [2, 1], 2)
1
([3, 2], [1, 1], [1, 1], 2)
0
([10, 10], [1, 1], [9, 9], 100)
277
([10, 10], [1, 1], [9, 9], 12)
1
([300, 275], [150, 150], [185, 100], 500)
9
([300, 275], [150, 150], [185, 185], 500)
8
